In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://www.marriott.com/hotel-search.mi"
driver.get(url)
time.sleep(1)


In [3]:
findreserve_button = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/div/section/div[1]/div/div/a')[0]
findreserve_button.click()
time.sleep(1)

In [4]:
brandfilter_button = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/section[2]/div[2]/div/div/span[4]')[0]
brandfilter_button.click()
time.sleep(1)

In [8]:
townplace_button = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/section[2]/div[2]/div/div/div/div[2]/div[1]/form/div[2]/div[1]/div/fieldset/div[4]/ul/li[2]/a/span')[0]
townplace_button.click()
time.sleep(1)

In [9]:
apply_button = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/section[2]/div[2]/div/div/div/div[2]/div[2]/div/div/div/form/button[1]')[0]
apply_button.click()
time.sleep(1)

In [10]:
counter = 1
listurls = []
while counter < 30:
    currenturl = driver.current_url
    print(f'Currently on page: {currenturl}.')
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    
#     nextbutton1
    hotelcards = soup.find('div', {'class' : 'js-property-list-container map-view-scroll-space'})
#     listurls = []
    try:
        for x in hotelcards.find_all('div', {'class' : 'l-row'}):
            try:
                href = x.find('a').attrs['href']

                if 'QuickView' in href:
                    url = {}
                    url['url'] = 'https://www.marriott.com/' + href
                    url['name'] = x.find('a').attrs['title']
                    url['code'] = x.find('a').attrs['data-marsha']
                    listurls.append(url)
        #             print(url)
        #             print()
        #             print('='*50)
        #             print()
            except:
                pass
    except:
        break
    nextbutton = soup.find_all('div', {'role' : 'navigation'})
    nextbutton1 = 'https://www.marriott.com/'
    for x in nextbutton:
        try:
            if counter == 1:
                button = x.find_all('a')[5].attrs['href']
                nextbutton1 = nextbutton1 + button 
            elif counter == 2 or counter == 11:
                button = x.find_all('a')[6].attrs['href']
                nextbutton1 = nextbutton1 + button
            elif counter == 3 or counter == 10:
                button = x.find_all('a')[7].attrs['href']
                nextbutton1 = nextbutton1 + button
            elif counter >= 4 and counter <= 9:
                button = x.find_all('a')[8].attrs['href']
                nextbutton1 = nextbutton1 + button
#             else:
#                 button = x.find_all('a')[5].attrs['href']
#                 nextbutton1 = nextbutton1 + button                
        except:
            continue
    driver.get(nextbutton1)
    counter += 1
listurls_df = pd.DataFrame(listurls)

Currently on page: https://www.marriott.com/search/filterSearch.mi.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=2.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=3.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=4.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=5.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=6.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=7.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=8.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=9.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=10.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=11.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=12.
Currently on page: https://www.marriott.com/default.mi.


In [11]:
listurls_df

,url,name,code
0,https://www.marriott.com//search/hotelQuickVie...,TownePlace Suites Aberdeen,ABRTS
1,https://www.marriott.com//search/hotelQuickVie...,TownePlace Suites Southern Pines Aberdeen,FAYSP
2,https://www.marriott.com//search/hotelQuickVie...,TownePlace Suites Abilene Northeast,ABITS
3,https://www.marriott.com//search/hotelQuickVie...,TownePlace Suites Thousand Oaks Agoura Hills,OXRAH
4,https://www.marriott.com//search/hotelQuickVie...,TownePlace Suites Aiken Whiskey Road,AGSTS
...,...,...,...
445,https://www.marriott.com//search/hotelQuickVie...,TownePlace Suites Wrentham Plainville,BOSTV
446,https://www.marriott.com//search/hotelQuickVie...,TownePlace Suites Grand Rapids Wyoming,GRRTW
447,https://www.marriott.com//search/hotelQuickVie...,TownePlace Suites York,HARYT
448,https://www.marriott.com//search/hotelQuickVie...,TownePlace Suites Newport News Yorktown,PHFTS


In [12]:
hotels = []
for index, row in listurls_df.iterrows():
    name = row['name'].replace(' ','-').replace('/','-').lower()
    code = row['code'].lower()
    url = 'https://www.marriott.com/hotels/travel/' + code + '-' + name
    hotel ={}
    hotel['name'] = name
    hotel['url'] = url
    hotels.append(hotel)
hotels_df = pd.DataFrame(hotels)

In [13]:
hotels_df

,name,url
0,towneplace-suites-aberdeen,https://www.marriott.com/hotels/travel/abrts-t...
1,towneplace-suites-southern-pines-aberdeen,https://www.marriott.com/hotels/travel/faysp-t...
2,towneplace-suites-abilene-northeast,https://www.marriott.com/hotels/travel/abits-t...
3,towneplace-suites-thousand-oaks-agoura-hills,https://www.marriott.com/hotels/travel/oxrah-t...
4,towneplace-suites-aiken-whiskey-road,https://www.marriott.com/hotels/travel/agsts-t...
...,...,...
445,towneplace-suites-wrentham-plainville,https://www.marriott.com/hotels/travel/bostv-t...
446,towneplace-suites-grand-rapids-wyoming,https://www.marriott.com/hotels/travel/grrtw-t...
447,towneplace-suites-york,https://www.marriott.com/hotels/travel/haryt-t...
448,towneplace-suites-newport-news-yorktown,https://www.marriott.com/hotels/travel/phfts-t...


In [14]:
%%time

hotel_list = []
counter = 0
for index, row in hotels_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'TownPlace'
        except:
            hotel['brand'] = 'TownPlace'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df       

{'brand': 'TownPlace', 'name': 'towneplace-suites-aberdeen', 'url': 'https://www.marriott.com/hotels/travel/abrts-towneplace-suites-aberdeen', 'street': '402 Norwood Street South', 'locality': 'Aberdeen', 'state': 'South Dakota', 'postalcode': '57401', 'coordinate1': '-98.444978669285', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ TownePlace Suites by Marriott® Aberdeen is an extended-stay hotel where you can balance work and life as you like, with the comfort, flexibility and affordability you require. Relax and feel at home with a fully equipped kitchen and high-speed Internet access. ]', 'overall_rating': 'N/A', 'cleanliness_rating': '4.7', 'service_rating': '4.7'}

2020-09-22 23:11:08.057584

Collected 1 hotels.
{'brand': 'TownPlace', 'name': 'towneplace-suites-southern-pines-aberdeen', 'url': 'https://www.marriott.com/hotels/travel/faysp-towneplace-suites-southern-pines-aberdeen', 'street': '205 Columbus Drive', 'locality': 'Aberdeen', 'state

{'brand': 'TownPlace', 'name': 'towneplace-suites-albany-downtown-medical-center', 'url': 'https://www.marriott.com/hotels/travel/albmc-towneplace-suites-albany-downtown-medical-center', 'street': '22 Holland Avenue', 'locality': 'Albany', 'state': 'New York', 'postalcode': '12209', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/albmc-towneplace-suites-albany-downtown-medical-center/?directPageRequest=true', 'coordinate2': '42.646018', 'coordinate3': '-73.772142', 'coordinate4': 'N/A', 'description': "[ Discover a homelike extended stay at TownePlace Suites Albany Downtown/Medical Center. Our hotel's prime location surrounds you with adventure. Explore the Empire State Plaza Convention Center, Times Union Center and the University of Albany. When it's time to unwind, discover soothing settings in our spacious suites. We deliver convenience with plush bedding and fully equipped kitchens. Business travelers can also benefit from ergonomic workstations and complimentary Wi-Fi

{'brand': 'TownPlace', 'name': 'towneplace-suites-alexandria-fort-belvoir', 'url': 'https://www.marriott.com/hotels/travel/wasln-towneplace-suites-alexandria-fort-belvoir', 'street': '8632 Woodlawn Court', 'locality': 'Alexandria', 'state': 'Virginia', 'postalcode': '22309', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/wasln-towneplace-suites-alexandria-fort-belvoir/?directPageRequest=true', 'coordinate2': '38.722219', 'coordinate3': '-77.123642', 'coordinate4': 'N/A', 'description': '[ The TownePlace Suites Alexandria Fort Belvoir is an all-suite, pet-friendly hotel located just a half mile from Fort Belvoir in Alexandria, VA and 3 miles to Mt. Vernon. We have everything you need to balance work and life during your short or long extended stay. Wake up every day to complimentary breakfast, serving both hot and cold items – eggs, bacon, sausage, assorted fruits, Danishes, cereals and more. Our grocery delivery and on-site pantry are both great for staying in for the even

{'brand': 'TownPlace', 'name': 'towneplace-suites-ames', 'url': 'https://www.marriott.com/hotels/travel/dsmta-towneplace-suites-ames', 'street': '1523 South Dayton Place', 'locality': 'Ames', 'state': 'Iowa', 'postalcode': '50010', 'coordinate1': '-93.5867', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ TownePlace Suites by Marriott® , in Ames, Ia. is an extended-stay hotel where you can balance work and life as you like, with the comfort, flexibility and affordability you require. Relax and feel at home with a fully equipped kitchen and high-speed Internet access. We are conveniently located at the crossroads of I-35 and Hwy 30.  Minutes away from Renewable Energy Group, Sauer-Danfoss, Syngenta, Barilla and Iowa State University. Local attractions are Jack Trice Stadium, Hilton Coliseum, Reiman Gardens, Boone &amp; Scenic Valley, Prairie Moon Winery &amp; Vinery ]', 'overall_rating': 'N/A', 'cleanliness_rating': '4.8', 'service_rating': '4.7'}

20

{'brand': 'TownPlace', 'name': 'towneplace-suites-atlanta-buckhead', 'url': 'https://www.marriott.com/hotels/travel/atlbu-towneplace-suites-atlanta-buckhead', 'street': '820 Sidney Marcus Boulevard NE', 'locality': 'Atlanta', 'state': 'Georgia', 'postalcode': '30324', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/atlbu-towneplace-suites-atlanta-buckhead/?directPageRequest=true', 'coordinate2': '33.825839', 'coordinate3': '-84.363129', 'coordinate4': 'N/A', 'description': "[ The TownePlace Suites by Marriott Atlanta- Buckhead is your home away from home.  Our extended stay all suites hotel is located in Atlanta's popular Buckhead neighborhood, yet just 4 miles from downtown Atlanta.  Each suite features a fully equipped kitchen with full-size refrigerator, microwave, dishwasher, and two stove-top burners.  Enjoy complimentary wireless internet, deluxe continental breakfast, local calls, parking, and guest laundry facilities.  New restaurants and shopping are adjacent to ou

...

{'brand': 'TownPlace', 'name': 'towneplace-suites-detroit-warren', 'url': 'https://www.marriott.com/hotels/travel/dtwdw-towneplace-suites-detroit-warren', 'street': '7601 Chicago Road', 'locality': 'Warren', 'state': 'Michigan', 'postalcode': '48092', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/dtwdw-towneplace-suites-detroit-warren/?directPageRequest=true', 'coordinate3': '42.523959', 'coordinate4': '-83.031038', 'description': "[ Come experience the completely renovated TownePlace Suites by Marriott® Detroit/Warren.  Chairman &amp; Platinum Award winners for Guest Satisfaction.  Conveniently located across from the GM Tech Center, within a few miles of TACOM, Chryslers' Warren/Sterling Heights Plant, General Dynamics and just 15 min. from the heart of Downtown. This all suites extended stay hotel is an economically smart choice for travelers on extended assignments, relocating to the area; or just in town for a few nights.  All suites have a 

{'brand': 'TownPlace', 'name': 'towneplace-suites-fort-lauderdale-weston', 'url': 'https://www.marriott.com/hotels/travel/flltw-towneplace-suites-fort-lauderdale-weston', 'street': '1545 Three Village Road', 'locality': 'Weston', 'state': 'Florida', 'postalcode': '33326', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/flltw-towneplace-suites-fort-lauderdale-weston/?directPageRequest=true', 'coordinate2': '26.098825', 'coordinate3': '-80.379631', 'coordinate4': 'N/A', 'description': '[ When your travels bring you to Florida, stay with us at TownePlace Suites by Marriott Fort Lauderdale Weston. We are in the heart of Weston, three minutes off I-75 at Exit 15 (Royal Palm Blvd). We are minutes away from Cleveland Clinic Hospital and Sawgrass Mills Outlet Mall. Watch your favorite teams play at BB&amp;T Center, home of the Florida Panthers or visit Hard Rock Stadium, home of the Miami Dolphins and Florida Marlins. A short drive gets you to our beautiful Ft. Lauderdale beaches. 

{'brand': 'TownPlace', 'name': 'towneplace-suites-burlington-williston', 'url': 'https://www.marriott.com/hotels/travel/btvts-towneplace-suites-burlington-williston', 'street': '66 Zephyr Road, Taft Corners', 'locality': 'Williston', 'state': 'Vermont', 'postalcode': '05495', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/btvts-towneplace-suites-burlington-williston/?directPageRequest=true', 'coordinate2': '44.453712', 'coordinate3': '-73.109838', 'coordinate4': 'N/A', 'description': '[ Enjoy a warm, Vermont experience at our TownePlace Suites Burlington extended stay hotel. Close to GlobalFoundries, Lake Champlain, downtown Burlington, the UVM Medical Center, and a number of government offices, our hotel is centrally located to business and local attractions. Our spacious suites equipped with full kitchens invite you to settle in and savor the comforts of home. Seamlessly maintain your essential routines with our state of the art fitness center, on site guest laundry and 

{'brand': 'TownPlace', 'name': 'towneplace-suites-wrentham-plainville', 'url': 'https://www.marriott.com/hotels/travel/bostv-towneplace-suites-wrentham-plainville', 'street': '65 Ledgeview Way', 'locality': 'Wrentham', 'state': 'Massachusetts', 'postalcode': '02093', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ TownePlace Suites by Marriott® is an extended-stay hotel where you can balance work and life as you like, with the comfort, flexibility and affordability you require. Relax and feel at home with a fully equipped kitchen and high-speed Internet access. ]', 'overall_rating': 'N/A', 'cleanliness_rating': 'N/A', 'service_rating': 'N/A'}

2020-09-22 23:28:47.662638

Collected 446 hotels.
{'brand': 'TownPlace', 'name': 'towneplace-suites-grand-rapids-wyoming', 'url': 'https://www.marriott.com/hotels/travel/grrtw-towneplace-suites-grand-rapids-wyoming', 'street': '5880 Clyde Park Avenue', 'locality': 'Wyoming', 'state': 'Mich

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,TownPlace,towneplace-suites-aberdeen,https://www.marriott.com/hotels/travel/abrts-t...,402 Norwood Street South,Aberdeen,South Dakota,57401,-98.444978669285,N/A,N/A,N/A,[ TownePlace Suites by Marriott® Aberdeen is a...,N/A,4.7,4.7
1,TownPlace,towneplace-suites-southern-pines-aberdeen,https://www.marriott.com/hotels/travel/faysp-t...,205 Columbus Drive,Aberdeen,North Carolina,28315,Reviews,https://www.marriott.com/hotels/maps/travel/fa...,35.156108,-79.421344,[ TownePlace Suites Southern Pines Aberdeen is...,N/A,4.9,4.9
2,TownPlace,towneplace-suites-abilene-northeast,https://www.marriott.com/hotels/travel/abits-t...,2141 Scottish Road,Abilene,Texas,79601,-99.688466,N/A,N/A,N/A,[ Business and leisure travelers will enjoy a ...,N/A,4.8,4.8
3,TownPlace,towneplace-suites-thousand-oaks-agoura-hills,https://www.marriott.com/hotels/travel/oxrah-t...,29505 Agoura Road,Agoura Hills,California,91301,N/A,N/A,N/A,N/A,[ The brand new TownePlace Suites by Marriott®...,N/A,N/A,N/A
4,TownPlace,towneplace-suites-aiken-whiskey-road,https://www.marriott.com/hotels/travel/agsts-t...,1008 Monterey Drive,Aiken,South Carolina,29803,33.5178561508655,-81.715244427323,N/A,N/A,[ TownePlace Suites Aiken Whiskey Road is Aike...,N/A,4.6,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,TownPlace,towneplace-suites-wrentham-plainville,https://www.marriott.com/hotels/travel/bostv-t...,65 Ledgeview Way,Wrentham,Massachusetts,02093,N/A,N/A,N/A,N/A,[ TownePlace Suites by Marriott® is an extende...,N/A,N/A,N/A
446,TownPlace,towneplace-suites-grand-rapids-wyoming,https://www.marriott.com/hotels/travel/grrtw-t...,5880 Clyde Park Avenue,Wyoming,Michigan,49509,N/A,N/A,N/A,N/A,[ TownePlace Suites by Marriott® is an extende...,N/A,N/A,N/A
447,TownPlace,towneplace-suites-york,https://www.marriott.com/hotels/travel/haryt-t...,2789 Concord Road,York,Pennsylvania,17402,39.982312,-76.670216,N/A,N/A,[ TownePlace Suites by Marriott® is an extende...,N/A,4.5,4.5
448,TownPlace,towneplace-suites-newport-news-yorktown,https://www.marriott.com/hotels/travel/phfts-t...,200 Cybernetics Way,Yorktown,Virginia,23693,https://www.marriott.com/hotels/maps/travel/ph...,37.117915,-76.467392,N/A,[ Just a few miles from a day of fun at the be...,N/A,4.4,4.5


In [15]:
hotel_df = hotel_df.drop_duplicates(subset='name')
hotel_df

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,TownPlace,towneplace-suites-aberdeen,https://www.marriott.com/hotels/travel/abrts-t...,402 Norwood Street South,Aberdeen,South Dakota,57401,-98.444978669285,N/A,N/A,N/A,[ TownePlace Suites by Marriott® Aberdeen is a...,N/A,4.7,4.7
1,TownPlace,towneplace-suites-southern-pines-aberdeen,https://www.marriott.com/hotels/travel/faysp-t...,205 Columbus Drive,Aberdeen,North Carolina,28315,Reviews,https://www.marriott.com/hotels/maps/travel/fa...,35.156108,-79.421344,[ TownePlace Suites Southern Pines Aberdeen is...,N/A,4.9,4.9
2,TownPlace,towneplace-suites-abilene-northeast,https://www.marriott.com/hotels/travel/abits-t...,2141 Scottish Road,Abilene,Texas,79601,-99.688466,N/A,N/A,N/A,[ Business and leisure travelers will enjoy a ...,N/A,4.8,4.8
3,TownPlace,towneplace-suites-thousand-oaks-agoura-hills,https://www.marriott.com/hotels/travel/oxrah-t...,29505 Agoura Road,Agoura Hills,California,91301,N/A,N/A,N/A,N/A,[ The brand new TownePlace Suites by Marriott®...,N/A,N/A,N/A
4,TownPlace,towneplace-suites-aiken-whiskey-road,https://www.marriott.com/hotels/travel/agsts-t...,1008 Monterey Drive,Aiken,South Carolina,29803,33.5178561508655,-81.715244427323,N/A,N/A,[ TownePlace Suites Aiken Whiskey Road is Aike...,N/A,4.6,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,TownPlace,towneplace-suites-wrentham-plainville,https://www.marriott.com/hotels/travel/bostv-t...,65 Ledgeview Way,Wrentham,Massachusetts,02093,N/A,N/A,N/A,N/A,[ TownePlace Suites by Marriott® is an extende...,N/A,N/A,N/A
446,TownPlace,towneplace-suites-grand-rapids-wyoming,https://www.marriott.com/hotels/travel/grrtw-t...,5880 Clyde Park Avenue,Wyoming,Michigan,49509,N/A,N/A,N/A,N/A,[ TownePlace Suites by Marriott® is an extende...,N/A,N/A,N/A
447,TownPlace,towneplace-suites-york,https://www.marriott.com/hotels/travel/haryt-t...,2789 Concord Road,York,Pennsylvania,17402,39.982312,-76.670216,N/A,N/A,[ TownePlace Suites by Marriott® is an extende...,N/A,4.5,4.5
448,TownPlace,towneplace-suites-newport-news-yorktown,https://www.marriott.com/hotels/travel/phfts-t...,200 Cybernetics Way,Yorktown,Virginia,23693,https://www.marriott.com/hotels/maps/travel/ph...,37.117915,-76.467392,N/A,[ Just a few miles from a day of fun at the be...,N/A,4.4,4.5


In [16]:
hotel_df.to_csv('townplace_description.csv')